In [ ]:
#approfondire le concern e fare file da condividere

In [47]:
import pymongo
import pprint
import json
import random
import datetime

# FUCTIONS

INSERT

In [48]:
def generate_array_to_insert(collection):
    example = {
    "id": 1,
    "name": "Pippo",
    "surname": "Pluto",
    "date_of_birth": "18/03/2000",
    "bank": "UniCredit",
    "balance": "10000",
    "test" : 1,
    }
    list_name = ['Sofia', 'Matteo', 'Giuseppe', 'Isabella', 'Federico', 'Giulia', 'Carlo', 'Veronica', 'Luca', 'Aurora', 'Marco', 'Alessia', 'Davide', 'Martina', 'Antonio', 'Ginevra', 'Simone', 'Camilla', 'Francesco', 'Sara', 'Andrea', 'Ludovica', 'Gabriele', 'Valentina', 'Enrico', 'Elisa', 'Davide', 'Benedetta', 'Alberto', 'Chiara', 'Paolo', 'Beatrice', 'Daniele', 'Eleonora', 'Stefano', 'Emanuela', 'Roberto', 'Caterina', 'Fabio', 'Alice', 'Tommaso', 'Cristina', 'Leonardo', 'Gaia', 'Filippo', 'Francesca', 'Ivan', 'Laura', 'Edoardo', 'Marta', 'Giovanni', 'Sofia', 'Riccardo', 'Ludovica', 'Mauro', 'Eleonora', 'Pietro', 'Margherita', 'Vittorio', 'Giulia', 'Enrico', 'Martina', 'Marco', 'Aurora', 'Andrea', 'Veronica', 'Alessandro', 'Ilaria', 'Dario', 'Silvia', 'Luca', 'Anna', 'Matteo', 'Federica', 'Carlo', 'Ludovico', 'Chiara', 'Fabrizio', 'Elisa', 'Antonio', 'Sara', 'Emanuele', 'Valentina', 'Gabriele', 'Benedetta', 'Simone', 'Camilla', 'Francesco', 'Martina', 'Giuseppe', 'Ludovica', 'Davide', 'Veronica', 'Andrea', 'Aurora', 'Marco', 'Giulia', 'Federico', 'Isabella', 'Matteo']
    list_surname = ["Rossi", "Bianchi", "Russo", "Esposito", "Ricci", "Ferrari", "Rizzo", "Romano", "Conti", "Santoro", "Marino", "Greco", "Neri", "Lombardi", "Bruno", "Moretti", "Costa", "Giordano", "Vitale", "Messina", "Farina", "Valenti", "Barbieri", "Serra", "Marini", "Caruso", "Lombardo", "Lombardi", "Gallo", "D'Angelo", "Pagano", "Ferrara", "Fabbri", "Monti", "Ferri", "Grasso", "Mancini", "Moretti", "Marchetti", "Cattaneo", "Costa", "Riva", "Ricciardi", "Pellegrini", "Amato", "Martini", "Basile", "De Luca", "Napolitano", "Fontana", "Carbone", "Grasso", "Bernardi", "Fabbiano", "Silvestri", "Palumbo", "Esposito", "Russo", "Colombo", "Sartori", "Donati", "Piras", "Bianco", "Coppola", "Rossetti", "Sanna", "Esposito", "Sorrentino", "Fontana", "Mazzoni", "Battaglia", "Serra", "Battaglia", "Pugliese", "Vitale", "Esposito", "Caruso", "Rizzo", "Ricci", "Marini", "De Rosa", "Marino", "Montanari", "Moretti", "Caruso", "Greco", "Ruggiero", "Ferretti", "Bianchi", "Giannini", "Giannini", "Rosso", "Basile", "Santoro", "De Angelis", "Ruggiero", "Pugliese", "Messina", "Barone", "Fontana"]
    italian_banks = [
    "Intesa Sanpaolo",
    "UniCredit",
    "Banco BPM",
    "Monte dei Paschi di Siena",
    "UBI Banca",
    "Banca Nazionale del Lavoro",
    "Banca Generali",
    "Mediobanca",
    "Cassa Depositi e Prestiti",
    "Banca Popolare di Sondrio"
    ]
    banks = []

    start_point = collection.find_one(sort=[("id", pymongo.DESCENDING)])
    start_point = start_point["id"]

    for i in range(start_point, start_point+1000):
        item = example.copy()

        rand_name = random.randint(0, 99)
        rand_year = random.randint(1960, 2005)
        rand_month = random.randint(1, 12)
        rand_day = random.randint(1, 31)
        rand_banca = random.randint(0,9)
        rand_balance = random.randint(1000, 1000000)


        item["id"] = i
        item["name"] = list_name[rand_name]
        item["surname"] = list_surname[rand_name]
        item["date_of_birth"] = f"{rand_day}/{rand_month}/{rand_year}"
        item["bank"] = italian_banks[rand_banca]
        item["balance"] = rand_balance
        item["date"] = datetime.datetime.now()
        item["test"] = 1

        banks.append(item)

    return banks

In [49]:
#insert sigle document multiple times
def insert_many_user_insert_one(session):
        coll = session.client.db_bank.bank
        value_to_insert = generate_array_to_insert(coll)
        app = []
        print("---")
        print(value_to_insert)
        print("---")
        for i in range(1, len(value_to_insert)):
            id_in = coll.insert_one(value_to_insert[i], session=session)
            app.append(id_in.inserted_id)
        return app

In [50]:
#insert multiple docs in a single command (bulk)
def insert_many_user_insert_many(session):
    value_to_insert = generate_array_to_insert()
    coll = session.client.db_bank.bank
    app = coll.insert_many(value_to_insert[i], session=session).inserted_ids
    return app

UPDATE

In [51]:
#single update
def wrapper_trans(id_snd, id_rcv, val):
    def callback(session):
        coll = session.client.db_bank.bank
        json_snd = coll.find_one({"id" : id_snd}, session = session)
        json_rcv = coll.find_one({"id" : id_rcv}, session = session)
        bal = json_snd["balance"]
        if bal >= val:
            new_bal_snd = json_snd["balance"] + val
            new_bal_rcv = json_snd["balance"] - val

            coll.update_one(
                {"_id": json_snd["_id"]},
                {"$set": {"balance": new_bal_snd}}, session = session)
            coll.update_one(
                {"_id": json_rcv["_id"]},
                {"$set": {"balance": new_bal_rcv}}, session = session)

            coll.update_one(
                {"_id": json_snd["_id"]},
                {"$push": {"trans": {
                    "id_user": json_rcv["_id"],
                    "type" : -1 ,
                    "value": val,
                    "date": datetime.datetime.now().strftime("%x")}}}, session = session)
            coll.update_one(
                {"_id": json_rcv["_id"]},
                {"$push": {"trans": {
                    "id_user": json_snd["_id"],
                    "type" : 1 ,
                    "value": val,
                    "date": datetime.datetime.now().strftime("%x")}}}, session = session)
        else:
            raise Exception("transaction rejecteded for insufficient money")
        final_snd = coll.find_one({"id" : id_snd}, session = session)
        final_rcv = coll.find_one({"id" : id_rcv}, session = session)
        app = [final_snd, final_rcv]
        return app

    return callback

In [52]:
update_users_transaction = wrapper_trans(1000007, 1000011, 1000)

FIND

In [55]:
#wrapper to find
def wrapper_bank_user(val):
    def callback(session):
        coll = session.client.db_bank.bank
        az = coll.find({"bank" : val}, session = session)
        app = []
        for a in az:
            app.append(a)
        return app
    return callback

In [56]:
find_bank_user = wrapper_bank_user("UniCredit")

In [57]:
def wrapper_date_transaction(date):
    def callback(session):
        coll = session.client.db_bank.bank
        az = coll.find({"trans.date" : date}, session = session)
        app = []
        for a in az:
            app.append(a)
        return app
    return callback

In [58]:
find_date_transaction = wrapper_date_transaction(datetime.datetime.now().strftime("%x"))

DELETE

In [59]:
def wrapper_remove(bank):
    def callback(session):
        coll = session.client.db_bank.bank
        found = coll.find({"bank": bank}, session = session)
        coll.delete_many({"bank": bank}, session = session)

        app = []
        for a in found:
            app.append(a)
        return app
    return callback

In [72]:
remove_bank_many = wrapper_remove("UBI Banca")

SESSION USING CALLBACK

In [61]:
def execute_func(client, callback, write_c, read_c, read_p): #Step 2: Start a client session.
    try:
        az = 0
        with client.start_session() as session:
            # Step 3: Use with_transaction to start a transaction, execute the callback, and commit (or abort on error).
            az = session.with_transaction(
                callback,
                #read_concern=ReadConcern("local"),
                read_concern = read_c,
                write_concern=write_c,
                #read_preference=ReadPreference.PRIMARY,
                read_preference = read_p
            )
    except Exception as ex:
        print("OS Exception:", ex)
    finally:
        print("it's ok")
        return az

# EXE

In [62]:

# For a replica set, include the replica set name and a seedlist of the members in the URI string; e.g.
# uriString = 'mongodb://mongodb0.example.com:27017,mongodb1.example.com:27017/?replicaSet=myRepl'
# For a sharded cluster, connect to the mongos instances; e.g.
# uriString = 'mongodb://mongos0.example.com:27017,mongos1.example.com:27017/'

uristring = "mongodb://localhost:27117"

client = pymongo.MongoClient(uristring)
#wc_majority = WriteConcern("majority", wtimeout=1000)
db = client.db_bank
collection = db.bank

In [63]:
#sti tre sono da modificare in base a che test si vuol fare
read_concern=pymongo.read_concern.ReadConcern("local")
write_concern=pymongo.write_concern.WriteConcern("majority")
read_preference=pymongo.ReadPreference.PRIMARY

In [64]:
execute_func(client, insert_many_user_insert_one, read_c = read_concern, write_c = write_concern, read_p = read_preference)

---
[{'id': 1000000, 'name': 'Fabio', 'surname': 'Marchetti', 'date_of_birth': '30/5/2003', 'bank': 'Banca Nazionale del Lavoro', 'balance': 632420, 'test': 1, 'date': datetime.datetime(2023, 7, 12, 11, 25, 24, 158139)}, {'id': 1000001, 'name': 'Valentina', 'surname': 'Montanari', 'date_of_birth': '6/10/2005', 'bank': 'Monte dei Paschi di Siena', 'balance': 365216, 'test': 1, 'date': datetime.datetime(2023, 7, 12, 11, 25, 24, 158139)}, {'id': 1000002, 'name': 'Tommaso', 'surname': 'Costa', 'date_of_birth': '7/10/1985', 'bank': 'Monte dei Paschi di Siena', 'balance': 507120, 'test': 1, 'date': datetime.datetime(2023, 7, 12, 11, 25, 24, 158139)}, {'id': 1000003, 'name': 'Matteo', 'surname': 'Fontana', 'date_of_birth': '25/3/2000', 'bank': 'Banco BPM', 'balance': 630116, 'test': 1, 'date': datetime.datetime(2023, 7, 12, 11, 25, 24, 158139)}, {'id': 1000004, 'name': 'Martina', 'surname': 'Piras', 'date_of_birth': '23/3/1989', 'bank': 'Banca Popolare di Sondrio', 'balance': 602581, 'test': 

it's ok


[ObjectId('64ae718457a8c8cbd2746483'),
 ObjectId('64ae718457a8c8cbd2746484'),
 ObjectId('64ae718457a8c8cbd2746485'),
 ObjectId('64ae718457a8c8cbd2746486'),
 ObjectId('64ae718457a8c8cbd2746487'),
 ObjectId('64ae718457a8c8cbd2746488'),
 ObjectId('64ae718457a8c8cbd2746489'),
 ObjectId('64ae718457a8c8cbd274648a'),
 ObjectId('64ae718457a8c8cbd274648b'),
 ObjectId('64ae718457a8c8cbd274648c'),
 ObjectId('64ae718457a8c8cbd274648d'),
 ObjectId('64ae718457a8c8cbd274648e'),
 ObjectId('64ae718457a8c8cbd274648f'),
 ObjectId('64ae718457a8c8cbd2746490'),
 ObjectId('64ae718457a8c8cbd2746491'),
 ObjectId('64ae718457a8c8cbd2746492'),
 ObjectId('64ae718457a8c8cbd2746493'),
 ObjectId('64ae718457a8c8cbd2746494'),
 ObjectId('64ae718457a8c8cbd2746495'),
 ObjectId('64ae718457a8c8cbd2746496'),
 ObjectId('64ae718457a8c8cbd2746497'),
 ObjectId('64ae718457a8c8cbd2746498'),
 ObjectId('64ae718457a8c8cbd2746499'),
 ObjectId('64ae718457a8c8cbd274649a'),
 ObjectId('64ae718457a8c8cbd274649b'),
 ObjectId('64ae718457a8c8

In [67]:
#example update
execute_func(client, update_users_transaction, read_c = read_concern, write_c = write_concern, read_p = read_preference)

it's ok


[{'_id': ObjectId('64ae718457a8c8cbd2746489'),
  'id': 1000007,
  'name': 'Enrico',
  'surname': 'Marini',
  'date_of_birth': '28/10/1962',
  'bank': 'UniCredit',
  'balance': 926677,
  'test': 1,
  'date': datetime.datetime(2023, 7, 12, 11, 25, 24, 158000),
  'trans': [{'id_user': ObjectId('64ae718457a8c8cbd274648d'),
    'type': -1,
    'value': 1000,
    'date': '07/12/23'}]},
 {'_id': ObjectId('64ae718457a8c8cbd274648d'),
  'id': 1000011,
  'name': 'Enrico',
  'surname': 'Donati',
  'date_of_birth': '8/2/1988',
  'bank': 'Banca Nazionale del Lavoro',
  'balance': 924677,
  'test': 1,
  'date': datetime.datetime(2023, 7, 12, 11, 25, 24, 158000),
  'trans': [{'id_user': ObjectId('64ae718457a8c8cbd2746489'),
    'type': 1,
    'value': 1000,
    'date': '07/12/23'}]}]

In [69]:
found = execute_func(client, find_bank_user, read_c = read_concern, write_c = write_concern, read_p = read_preference)
print(len(found))

it's ok
100266


In [70]:
found = execute_func(client, find_date_transaction, read_c = read_concern, write_c = write_concern, read_p = read_preference)
print(found)

it's ok
[{'_id': ObjectId('64ae718457a8c8cbd2746489'), 'id': 1000007, 'name': 'Enrico', 'surname': 'Marini', 'date_of_birth': '28/10/1962', 'bank': 'UniCredit', 'balance': 926677, 'test': 1, 'date': datetime.datetime(2023, 7, 12, 11, 25, 24, 158000), 'trans': [{'id_user': ObjectId('64ae718457a8c8cbd274648d'), 'type': -1, 'value': 1000, 'date': '07/12/23'}]}, {'_id': ObjectId('64ae718457a8c8cbd274648d'), 'id': 1000011, 'name': 'Enrico', 'surname': 'Donati', 'date_of_birth': '8/2/1988', 'bank': 'Banca Nazionale del Lavoro', 'balance': 924677, 'test': 1, 'date': datetime.datetime(2023, 7, 12, 11, 25, 24, 158000), 'trans': [{'id_user': ObjectId('64ae718457a8c8cbd2746489'), 'type': 1, 'value': 1000, 'date': '07/12/23'}]}]


In [73]:
execute_func(client, remove_bank_many, read_c = read_concern, write_c = write_concern, read_p = read_preference)

it's ok


[]